In [1]:
import geopandas as gpd
import pandas as pd
import warnings
from shapely.geometry import Point

# Apaga todas las advertencias (warnings)
warnings.filterwarnings("ignore")

In [9]:
nodes = gpd.read_file('/app/data/red/nodes')

In [3]:
edges = gpd.read_file('/app/data/red/edges')

In [14]:
### Para el caso de una red totalmente nueva usar --->
nodes.rename(columns={'OBJECTID': 'osmid'}, inplace=True)

### Para el caso de editar la red existente usar --->
# nodes['temp_id'] = nodes.index
# nodes['str_id'] = nodes['temp_id'].astype(str)
# nodes.loc[nodes['osmid']==0, 'osmid'] = '992023' + nodes['str_id']
# nodes.drop(columns=['temp_id', 'str_id'], inplace=True)

# nodes.x = nodes.geometry.x
# nodes.y = nodes.geometry.y
# net_nodes = nodes.copy()
# net_nodes.drop(columns=['x', 'y'], inplace=True)

In [25]:
### Para el caso de una red totalmente nueva -->
edge_cols = ['key', 'osmid', 'name', 'geometry']
new_edges = edges[edge_cols]
new_edges['u'] = pd.NA
new_edges['v'] = pd.NA

### Para el caso de editar la red existente usar --->
# Seleccionamos solos los nuevos vertices (osmid que empieza con 2023)
edges_mask = edges['osmid'].astype(str).str.startswith("2023")
# edges_mask = edges_mask & (edges['length'] > 0)
edges_cols = ['osmid', 'length', 'geometry']
new_edges = edges.loc[edges_mask, edges_cols]
# new_edges

In [26]:
# Añade una nueva columna llamada "begin" que contiene el primer punto de cada línea
new_edges['begin'] = new_edges['geometry'].apply(lambda geom: Point(geom.coords[0]))

# Añade una nueva columna llamada "end" que contiene el último punto de cada línea
new_edges['end'] = new_edges['geometry'].apply(lambda geom: Point(geom.coords[-1]))

# Función para encontrar el punto más cercano en el GeoDataFrame "nodes"
def find_nearest_node(point):
    nearest = nodes['geometry'].apply(lambda x: point.distance(x)).idxmin()
    return nodes.loc[nearest, 'osmid']

# Aplicamos la función para encontrar el "source" y "destination" para cada línea en "new_edges"
new_edges['u'] = new_edges['begin'].apply(find_nearest_node)
new_edges['v'] = new_edges['end'].apply(find_nearest_node)

In [33]:
new_edges['length'] = new_edges.geometry.length

In [36]:
edges_cols = ['u', 'v', 'osmid', 'length', 'geometry']

try:
    old_edges = edges.loc[~edges_mask, edges_cols]
    new_edges = new_edges[edges_cols]
    net_edges = pd.concat([old_edges, new_edges])
except:
    net_edges = new_edges[edges_cols]

In [39]:
net_nodes = nodes.copy()
net_nodes['x'] = net_nodes['geometry'].x
net_nodes['y'] = net_nodes['geometry'].y
net_nodes['lon'] = net_nodes['x']
net_nodes['lat'] = net_nodes['y']

In [40]:
def format_net_nodes(nodes_df):
    nodes = pd.DataFrame(
        {
            'osmid': nodes_df['osmid'].astype(int),
            'lat' : nodes_df.geometry.y.astype(float),
            'lon' : nodes_df.geometry.x.astype(float),
            'y' : nodes_df.geometry.y.astype(float),
            'x' : nodes_df.geometry.x.astype(float),
        }
    )
    nodes['id'] = nodes['osmid'].values

    nodes = gpd.GeoDataFrame(data=nodes, geometry=nodes_df.geometry)
    nodes.set_index('osmid', inplace=True)
    return nodes

nodes = format_net_nodes(net_nodes)

In [41]:
def format_net_edges(edges_df):
    edges = pd.DataFrame(
        {
            'u': edges_df['u'].astype(int),
            'v': edges_df['v'].astype(int),
            'from': edges_df['u'].astype(int),
            'to': edges_df['v'].astype(int),
            'osmid': edges_df['osmid'].astype(int),
            'length': edges_df['length'].astype(float)
        }
    )
    edges['key'] = 0
    edges['key'] = edges['key'].astype(int)
    edges = gpd.GeoDataFrame(data=edges, geometry=edges_df.geometry)
    edges.set_index(['u', 'v', 'key'], inplace=True)
    return edges

edges = format_net_edges(net_edges)

In [42]:
mask = (~edges['to'].isin(nodes['id']))|(~edges['from'].isin(nodes['id']))
edges_ok = edges[~mask]
edges_to_fix = edges[mask].reset_index(drop=True)

In [44]:
# Añade una nueva columna llamada "begin" que contiene el primer punto de cada línea
edges_to_fix['begin'] = edges_to_fix['geometry'].apply(lambda geom: Point(geom.coords[0]))

# Añade una nueva columna llamada "end" que contiene el último punto de cada línea
edges_to_fix['end'] = edges_to_fix['geometry'].apply(lambda geom: Point(geom.coords[-1]))

# Función para encontrar el punto más cercano en el GeoDataFrame "nodes"
def find_nearest_node(point):
    nearest = nodes['geometry'].apply(lambda x: point.distance(x)).idxmin()
    return nodes.loc[nearest, 'id']

# Aplicamos la función para encontrar el "source" y "destination" para cada línea en "new_edges"
edges_to_fix['u'] = edges_to_fix['begin'].apply(find_nearest_node)
edges_to_fix['v'] = edges_to_fix['end'].apply(find_nearest_node)

In [45]:
edges_fixed = format_net_edges(edges_to_fix)

In [46]:
out_edges = pd.concat([edges_ok, edges_fixed])

In [47]:
out_nodes = nodes.copy()

In [48]:
import pandana as pdna

nodes_df = out_nodes
edges_df = out_edges

net = pdna.Network(
    nodes_df['lon'],
    nodes_df['lat'],
    edges_df['from'],
    edges_df['to'],
    edges_df[['length']]
)
# Si llegaste hasta aqui sin errores, misión cumplida

Generating contraction hierarchies with 16 threads.
Setting CH node vector of size 36885
Setting CH edge vector of size 81542
Range graph removed 81746 edges of 163084
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


In [53]:
net_edges['key'] = 0

In [49]:
out_nodes.to_file('/app/data/output/streets/future/nodes')
out_edges.to_file('/app/data/output/streets/future/edges')

In [59]:
out_nodes.reset_index().to_parquet('/app/assets/streets/future/nodes.parquet')
out_edges.reset_index().to_parquet('/app/assets/streets/future/edges.parquet')

In [58]:
out_edges.reset_index()

,u,v,key,from,to,osmid,length,geometry
0,20970,20953,0,20970,20953,368010828,4.812181,"LINESTRING (674769.156 5922560.301, 674767.130..."
1,20970,20987,0,20970,20987,368010828,3.901331,"LINESTRING (674769.156 5922560.301, 674771.258..."
2,20953,20912,0,20953,20912,367537909,5.850963,"LINESTRING (674767.130 5922555.936, 674761.516..."
3,20953,20945,0,20953,20945,368010828,6.686680,"LINESTRING (674767.130 5922555.936, 674766.174..."
4,20953,20970,0,20953,20970,368010828,4.812181,"LINESTRING (674767.130 5922555.936, 674769.156..."
...,...,...,...,...,...,...,...,...
81297,5957,6314,0,5957,6314,98795642,110.834694,"LINESTRING (672949.506 5921312.795, 672999.561..."
81298,6384,6476,0,6384,6476,412188927,18.178679,"LINESTRING (673009.590 5921219.877, 673025.863..."
81299,6476,6384,0,6476,6384,412188927,18.178679,"LINESTRING (673025.863 5921227.978, 673009.590..."
81300,6624,6554,0,6624,6554,79820165,14.652808,"LINESTRING (673057.985 5921370.605, 673045.064..."


In [ ]:
out_nodes = gpd.read_parquet('/app/assets/streets/future/nodes.parquet')
out_edges = gpd.read_parquet('/app/assets/streets/future/edges.parquet')

In [ ]:
out_nodes.to_file('/app/data/output/streets/future/nodes')
out_edges.to_file('/app/data/output/streets/future/edges')

In [75]:
edges = gpd.read_parquet('/app/data/red_ajuste_final_final_ahorasiquesi/edges.parquet')
quadp = gpd.read_parquet('/app/data/red_ajuste_final_final_ahorasiquesi/cuadrante_prueba.parquet')

edges_cols = ['u', 'v', 'key', 'osmid', 'geometry']
edges = edges[edges_cols]

area_edges = gpd.sjoin(edges, quadp, how='inner', op='intersects')
area_edges = area_edges[edges_cols]

In [77]:
area_edges['u']

,u,v,key,osmid,geometry
40,3.034822e+08,9.731293e+09,0.0,1.101648e+08,"LINESTRING (673375.289 5921331.032, 673374.368..."
41,3.034822e+08,9.665359e+09,0.0,1.038108e+09,"LINESTRING (673375.289 5921331.032, 673378.662..."
42,3.034822e+08,9.665359e+09,0.0,1.088073e+09,"LINESTRING (673375.289 5921331.032, 673372.843..."
86,3.035266e+08,1.285932e+09,0.0,5.676435e+08,"LINESTRING (673019.210 5921086.532, 673032.953..."
125,3.098928e+08,1.778731e+09,0.0,1.745759e+08,"LINESTRING (673066.474 5921968.102, 673093.130..."
...,...,...,...,...,...
81374,1.142710e+09,1.541918e+09,0.0,9.876899e+07,"LINESTRING (673275.616 5920704.755, 673287.662..."
81375,1.541918e+09,1.142710e+09,0.0,9.876899e+07,"LINESTRING (673287.662 5920709.589, 673275.616..."
81376,1.148111e+09,1.825504e+09,0.0,9.925475e+07,"LINESTRING (673399.535 5920600.817, 673389.622..."
81377,1.825504e+09,1.148111e+09,0.0,9.925475e+07,"LINESTRING (673350.269 5920578.234, 673356.686..."


In [78]:
from shapely.geometry import LineString

edges_df = area_edges.copy()
print(edges_df.shape)

# Función para invertir la geometría y mantener el formato LINESTRING
def reverse_geometry(geometry):
    return LineString(geometry.coords[::-1])

# Asegúrate de que la columna 'u' siempre sea menor que 'v'
edges_df['sorted_edges'] = edges_df.apply(lambda row: tuple(sorted([row['u'], row['v']])), axis=1)

duplicated_pairs = edges_df[edges_df.duplicated(subset='sorted_edges', keep=False)]

edges_df.drop_duplicates(subset=['sorted_edges'], inplace=True, keep=False)
print(edges_df.shape)

reverse_edges = edges_df[['key', 'osmid', 'geometry']]
reverse_edges['geometry'] = reverse_edges['geometry'].apply(reverse_geometry)
reverse_edges[['u', 'v']]  = [pd.NA, pd.NA]
# reverse_edges[['u', 'v']]  = edges_df[['v', 'u']]
reverse_edges.shape

cols = ['u', 'v', 'key', 'osmid', 'geometry']
edges_df = pd.concat([edges_df, reverse_edges])
edges_df

(4486, 5)
(7, 6)


,u,v,key,osmid,geometry,sorted_edges
15206,1.438854e+09,9.951601e+09,0.0,166769811.0,"LINESTRING (673033.523 5922036.999, 673020.683...","(1438854387.0, 9951601269.0)"
16930,1.778766e+09,7.015760e+09,0.0,166431437.0,"LINESTRING (673210.143 5921711.933, 673204.868...","(1778765546.0, 7015759650.0)"
16937,1.778766e+09,1.778766e+09,0.0,166431438.0,"LINESTRING (673247.529 5921729.215, 673215.030...","(1778765552.0, 1778765554.0)"
21161,2.224337e+09,1.112920e+10,0.0,212592950.0,"LINESTRING (673240.922 5921742.591, 673204.868...","(2224336613.0, 11129202929.0)"
51890,7.873477e+09,7.873477e+09,0.0,844051104.0,"LINESTRING (673084.993 5921929.317, 673081.783...","(7873476942.0, 7873476944.0)"
58671,9.140777e+09,9.140777e+09,0.0,98795720.0,"LINESTRING (673111.799 5921517.424, 673072.399...","(9140777077.0, 9140777078.0)"
60512,9.694562e+09,7.816091e+09,0.0,988969778.0,"LINESTRING (672859.158 5921560.490, 672876.747...","(7816090509.0, 9694561972.0)"
15206,NaN,NaN,0.0,166769811.0,"LINESTRING (673020.683 5922031.495, 673033.523...",NaN
16930,NaN,NaN,0.0,166431437.0,"LINESTRING (673193.828 5921751.606, 673204.868...",NaN
16937,NaN,NaN,0.0,166431438.0,"LINESTRING (673215.030 5921714.469, 673247.529...",NaN
